In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoTokenizer, AutoModel, BartForConditionalGeneration, BartTokenizer, pipeline
import torch
import nltk
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
# Load the extracted text from Google Drive
extracted_text_file = '/content/drive/MyDrive/Patient Tests/extracted_text.txt'  # Adjust the path if needed

with open(extracted_text_file, 'r') as file:
    clinical_notes = file.read()

inputs = tokenizer(clinical_notes, return_tensors="pt", max_length=512, truncation=True)

In [10]:
with torch.no_grad():
    outputs = model(**inputs)
    clinical_embeddings = outputs.last_hidden_state.mean(dim=1)

In [11]:
summarization_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
summarization_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
summary_ids = summarization_model.generate(
    input_ids=inputs['input_ids'],
    max_length=100,  # Adjusted maximum length
    min_length=30,  # Adjusted minimum length
    num_beams=4,
    early_stopping=True
)

# Decode the summary with BART's tokenizer
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [13]:
# Clean the summary
summary = re.sub(r'\[unused\d+\]', '', summary)
summary = re.sub(r'[^a-zA-Z0-9\s.,-]', '', summary)
summary = re.sub(r'\s+', ' ', summary)
summary = summary.strip()

# Nltk for sentence tokenization
summary_sentences = nltk.tokenize.sent_tokenize(summary)

# Deduplicate sentences
unique_summary_sentences = list(dict.fromkeys(summary_sentences))
cleaned_summary = ' '.join(unique_summary_sentences)

cleaned_summary = re.sub(r'\s+', ' ', cleaned_summary).strip()

# Replace any occurrence of " a " (with spaces) to avoid odd spacing
cleaned_summary = re.sub(r'\s+a\s+', ' ', cleaned_summary)

print("Cleaned Summary:", cleaned_summary)

Cleaned Summary: a 45 - year - old female who reports experiencing frequent migraines approximately 3 - 4 times week. each episode lasts between 4 to 6 hours and is accompanied by nausea and light sensitivity. additionally, she has experienced episodes of dizziness that occur sporadically, sometimes triggering falls. the patient denies any recent head trauma but states that she has been under significant stress due to work. past medical history includes hypertension, diagnosed 5 years
